In [47]:
import DatastreamDSWS as dsws
from math import isnan

In [89]:
def data_prep(df):
    price_dict = df.to_dict()
    price_dict = dict((key[0], value) for (key, value) in price_dict.items())
    for date in price_dict:
        price_dict[date] = dict((key, value) for (key, value) in price_dict[date].items() if not isnan(value))
    return(price_dict)

def cal_mdd(price_dict):
    # (1) create Python dictionary to store MDD of each instrument
    mdd = {}
    for _ in price_dict:
        # (2) declare variable to store min and max price of each range
        max_price, min_price = -1, -1
        # (3) create Python list to store drawdown of each range
        mdds = []
        for date in price_dict[_]:
            #print(str(date)+": "+str(price_dict[_][date]))
            price = price_dict[_][date]
            # (4) set new min/max price if this price is higher than current max price
            if price > max_price or max_price == -1:
                if min_price != -1 and max_price != -1:
                    # (5) calculate drawdown % of the previous min-max
                    mdds.append((min_price-max_price)/max_price*100)
                max_price = price
                min_price = -1
            # (6) set only new min price if this price is lower than current min price
            if price < min_price or min_price == -1:
                min_price = price
        # (7) calculate drawdown % of the latest min-max
        mdds.append((min_price-max_price)/max_price*100)
        # (8) store maximum drawdown into Python dictionary with its instrument as a key
        ## use min() function as the drawdowns are less than zero
        mdd[_] = min(mdds)
    return(mdd)

In [63]:
ds = dsws.Datastream(username = '### Datastream username ###', password = '### Datastream password ###')

In [64]:
df = ds.get_data(tickers = 'LP68511418,LP68087222,LP68663339', fields = ['X']
                 , start = '-12M', end = '0M', freq='D')
display(df)

Instrument,LP68511418,LP68087222,LP68663339
Field,X,X,X
Currency,E,E,CH
Dates,,,
2021-07-20,122.60,222.53,1.00
2021-07-21,124.75,224.24,1.00
2021-07-22,125.74,225.11,1.00
2021-07-23,127.23,227.19,1.00
2021-07-26,127.16,226.29,1.00
...,...,...,...
2022-07-14,107.47,216.68,1.09


In [90]:
price_dict = data_prep(df)
price_dict

{'LP68511418': {'2021-07-20': 122.6,
  '2021-07-21': 124.75,
  '2021-07-22': 125.74,
  '2021-07-23': 127.23,
  '2021-07-26': 127.16,
  '2021-07-27': 126.08,
  '2021-07-28': 127.14,
  '2021-07-29': 127.71,
  '2021-07-30': 126.92,
  '2021-08-02': 127.73,
  '2021-08-03': 127.93,
  '2021-08-04': 128.73,
  '2021-08-05': 129.33,
  '2021-08-06': 129.7,
  '2021-08-09': 129.84,
  '2021-08-10': 130.25,
  '2021-08-11': 130.73,
  '2021-08-12': 131.21,
  '2021-08-13': 131.41,
  '2021-08-16': 130.59,
  '2021-08-17': 130.31,
  '2021-08-18': 130.45,
  '2021-08-19': 128.47,
  '2021-08-20': 128.9,
  '2021-08-23': 129.81,
  '2021-08-24': 129.94,
  '2021-08-25': 130.03,
  '2021-08-26': 129.58,
  '2021-08-27': 130.12,
  '2021-08-30': 130.28,
  '2021-08-31': 130.04,
  '2021-09-01': 131.04,
  '2021-09-02': 131.27,
  '2021-09-03': 130.35,
  '2021-09-06': 131.36,
  '2021-09-07': 131.03,
  '2021-09-08': 129.72,
  '2021-09-09': 129.66,
  '2021-09-10': 129.29,
  '2021-09-13': 129.7,
  '2021-09-14': 129.84,
  '202

In [91]:
mdd_last_1_year = cal_mdd(price_dict)
mdd_last_1_year

{'LP68511418': -21.79250166383198,
 'LP68087222': -14.638469783164323,
 'LP68663339': -36.206896551724135}

### What can you do with MDDs?
Let's look at an example of using MDD as a factor to consider investing in any instruments, I'm going to do the backtesting by looking at the yearly MDD of the previous year and select funds with less MDD to be invested, then check the result by checking their return of the previous year to see if it helps me to loss less.

In [130]:
# Retrieve data from Datastream
instruments = 'LP40221832,LP40109383,LP40229598,LP40229602,LP40228661,LP40048702,LP40221655,LP40005806'

df_prev_year = ds.get_data(tickers = instruments, fields = ['X']
                 , start = '-24M', end = '-12M', freq='D')
price_dict = data_prep(df_prev_year)
mdd_prev_year = cal_mdd(price_dict)

In [131]:
mdd_prev_year

{'LP40221832': -8.717978107627955,
 'LP40109383': -11.370786516853938,
 'LP40229598': -10.834371108343701,
 'LP40229602': -19.906166219839147,
 'LP40228661': -2.4832855778414498,
 'LP40048702': -2.2889842632331923,
 'LP40221655': -16.933066933066936,
 'LP40005806': -8.81461598406788}

In [ ]:
# Check with Eikon
import as ek

ek.set_app_key('### YOUR EIKON APP KEY ###')

In [132]:
year_return, err = ek.get_data(['LP40221832','LP40109383','LP40229598','LP40229602','LP40228661','LP40048702','LP40221655','LP40005806'],
                      ['TR.TotalReturn1YrCrossAsset'])
year_return_dict = dict(zip(year_return['Instrument'], year_return['1 Year Total Return Cross Asset']))

In [140]:
year_return_dict

{'LP40221832': -23.2527537,
 'LP40109383': -32.323344,
 'LP40229598': -32.2921591,
 'LP40229602': -35.0363303,
 'LP40228661': -11.3077075,
 'LP40048702': -14.267095,
 'LP40221655': -32.3474739,
 'LP40005806': -19.5482169}

### Conclusion
Last but not least, The Maximum Drawdown (MDD) is an indicator of downside risk over a specific time period, which is daily data of the last 1 year in this case. This data can be used to check the fund's past performance to be a part of the factors to consider investing in it. I hope you have fun with this article and if you have any questions or issues regarding API usage, please don’t hesitate to post your question on our Q&A forum

### Reference
 - [DataStream Web Service (including Overview, Quickstart guide, Documentation, Useful resources provided for downloading, tutorials)](https://developers.refinitiv.com/en/api-catalog/eikon/datastream-web-service)
 - [DataStream datatype search tool](https://developers.refinitiv.com/en/api-catalog/eikon/datastream-web-service)

### Thank you :)